<a href="https://colab.research.google.com/github/Mays-Waddah/CV-Segmenter-/blob/main/CV_Segmenter_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pdfplumber
!pip install spacy
!pip install transformers
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 43.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import pdfplumber

# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text()
    return text

# Example usage
pdf_path = 'content/sample_cv.pdf'  # Path to your PDF file in Google Colab
extracted_text = extract_text_from_pdf(pdf_path)
print(extracted_text)

JohnDoe
Phone:+1234567890|Email:john.doe@email.com
LinkedIn:linkedin.com/in/johndoe
Objective
SeekingasoftwareengineeringpositionatadynamiccompanywhereIcanleverage
myskillsindevelopment,problem-solving,andsystemautomation.
WorkandEmployment
SoftwareEngineer
XYZTechnologies,June2019-Present
-Leddevelopmentofcloud-basedapplicationsusingPythonandDocker.
-ImplementedCI/CDpipelineswithJenkinsandautomatedtesting.
-Collaboratedwithcross-functionalteamstooptimizesystemperformance.
JuniorDeveloper
ABCCorp,July2017-May2019
-AssistedindevelopingwebapplicationsusingJavaScriptandReact.
-WroteSQLqueriestomanageandretrievedataforreporting.
-DevelopedautomatedscriptsinPythontoenhancethedeploymentprocess.
EducationandTraining
B.Sc.inComputerScience
UniversityofExample,Graduated2017
-RelevantCoursework:DataStructures,Algorithms,OperatingSystems
Skills
-Programming:Python,JavaScript,C#,SQL
-Tools:Docker,Jenkins,Git,AWS,Kubernetes
-Other:AgileDevelopment,UnitTesting,CI/CD,RESTAPIs
Accomplishments
-AWSCert

Use spaCy to preprocess the text by tokenizing and cleaning it. This step will prepare the text for further classification and segmentation.


In [ ]:
import spacy

# Load the small English model from spaCy
nlp = spacy.load("en_core_web_sm")

# Preprocess the extracted text
def preprocess_text(text):
    doc = nlp(text)
    # Tokenize, lemmatize and remove stop words
    clean_text = [token.lemma_ for token in doc if not token.is_stop]
    return ' '.join(clean_text)

cleaned_text = preprocess_text(extracted_text)
print(cleaned_text)

JohnDoe 
 Phone:+1234567890|Email:john.doe@email.com 
 LinkedIn : linkedin.com / / johndoe 
 Objective 
 SeekingasoftwareengineeringpositionatadynamiccompanywhereIcanleverage 
 myskillsindevelopment , problem - solving , andsystemautomation . 
 WorkandEmployment 
 SoftwareEngineer 
 xyztechnologie , June2019 - Present 
 -leddevelopmentofcloud - basedapplicationsusingpythonanddocker . 
 -ImplementedCI / cdpipelineswithjenkinsandautomatedtesting . 
 -collaboratedwithcross - functionalteamstooptimizesystemperformance . 
 JuniorDeveloper 
 ABCCorp , July2017 - May2019 
 -AssistedindevelopingwebapplicationsusingJavaScriptandReact . 
 -wrotesqlqueriestomanageandretrievedataforreporting . 
 -developedautomatedscriptsinpythontoenhancethedeploymentprocess . 
 EducationandTraining 
 b.sc.incomputerscience 
 UniversityofExample , Graduated2017 
 -RelevantCoursework : datastructure , Algorithms , OperatingSystems 
 skill 
 -programming : Python , JavaScript , C#,SQL 
 -tool : Docker , Jenkins , Gi

Here, I’ll demonstrate how to use a pre-trained BERT model with Hugging Face’s transformers library. We'll classify each sentence or block into one of the CV sections.

In [ ]:
from transformers import pipeline

# Use a pre-trained BERT model for text classification
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Define the possible sections in a CV
cv_sections = ["Objective", "Work and Employment", "Education and Training", "Skills", "Accomplishments"]

# Function to classify segments of text
def classify_sections(text_blocks):
    sections = {}
    for block in text_blocks:
        result = classifier(block, candidate_labels=cv_sections)
        top_label = result['labels'][0]
        if top_label not in sections:
            sections[top_label] = []
        sections[top_label].append(block)
    return sections

# Example: Break the cleaned text into sentences/paragraphs
text_blocks = cleaned_text.split('\n\n')  # Splitting by double newlines (paragraphs)

# Classify the sections of the CV
cv_sections_classified = classify_sections(text_blocks)

# Display the classified sections
for section, content in cv_sections_classified.items():
    print(f"--- {section} ---")
    print('\n'.join(content))

--- Work and Employment ---
JohnDoe 
 Phone:+1234567890|Email:john.doe@email.com 
 LinkedIn : linkedin.com / / johndoe 
 Objective 
 SeekingasoftwareengineeringpositionatadynamiccompanywhereIcanleverage 
 myskillsindevelopment , problem - solving , andsystemautomation . 
 WorkandEmployment 
 SoftwareEngineer 
 xyztechnologie , June2019 - Present 
 -leddevelopmentofcloud - basedapplicationsusingpythonanddocker . 
 -ImplementedCI / cdpipelineswithjenkinsandautomatedtesting . 
 -collaboratedwithcross - functionalteamstooptimizesystemperformance . 
 JuniorDeveloper 
 ABCCorp , July2017 - May2019 
 -AssistedindevelopingwebapplicationsusingJavaScriptandReact . 
 -wrotesqlqueriestomanageandretrievedataforreporting . 
 -developedautomatedscriptsinpythontoenhancethedeploymentprocess . 
 EducationandTraining 
 b.sc.incomputerscience 
 UniversityofExample , Graduated2017 
 -RelevantCoursework : datastructure , Algorithms , OperatingSystems 
 skill 
 -programming : Python , JavaScript , C#,SQL 
 -

The model "facebook/bart-large-mnli" refers to a pre-trained BART (Bidirectional and Auto-Regressive Transformers) model developed by Facebook. The model is trained for natural language understanding tasks, specifically multi-label classification. It's commonly used for tasks like summarization and classification.

Regex can be used to add precision in segmenting specific sections, such as Work and Employment or Education and Training, based on patterns like dates or specific keywords.

In [ ]:
import re

def segment_with_regex(text):
    sections = {
        'Work and Employment': re.findall(r'(\d{4}.*?present|(\d{4}-\d{4}).*?title)', text, re.DOTALL),
        'Education and Training': re.findall(r'(B\.Sc\.|M\.Sc\.|Ph\.D\.|degree)', text),
        'Skills': re.findall(r'(Skills|Languages|Programming)', text),
    }
    return sections

regex_segments = segment_with_regex(extracted_text)
print(regex_segments)

{'Work and Employment': [], 'Education and Training': ['B.Sc.'], 'Skills': ['Skills', 'Programming']}


In [ ]:
# Combine classified sections with regex segments
final_cv_structure = {**cv_sections_classified, **regex_segments}

# Print the final structure of the CV
for section, content in final_cv_structure.items():
    print(f"--- {section} ---")
    print(content)

--- Work and Employment ---
[]
--- Education and Training ---
['B.Sc.']
--- Skills ---
['Skills', 'Programming']
